In [1]:
from keras.preprocessing.image import load_img , img_to_array
from prettytable import PrettyTable
import numpy as np
import os 

def generate_data(path):
    """Load image data from the root directory.
    
    Load image data from the root directory and convert it 
    to numpy array format, automatically generate data and 
    its corresponding labels.
    
    # Parameters
        path : str
            path of the data directory
            in the format like:
            path/
                dogs/
                    dog001.jpg
                    dog002.jpg
                    ...
                cats/
                    cat001/jpg
                    cat002.jpg
                    ...
                elephants/
                    elephant001.jpg
                    elephant002.jpg
                    ...
    # Returns
        numpy array tuple: (data, labels)
    """
    files = os.listdir(path)
    data = []
    labels = []
    nb_classes = len(files)
    class_name = []
    class_list = []
    
    for i in range(nb_classes):
        sub_path = path + '\\' + files[i]  
        pics = os.listdir(sub_path)
        nb_pics = len(pics)
        file_name = files[i]
        class_name.append(file_name)
        class_list.append(nb_pics)
        for j in range(nb_pics):
            img_path = sub_path + '\\' + pics[j]
            img = load_img(img_path)
            x = img_to_array(img)
            data.append(x)
            labels.append(i)
    
    data = np.array(data)
    labels = np.array(labels)
    print(sum(class_list), "samples in", nb_classes, "categories")
    print("The shape of each sample is", x.shape)
    table = PrettyTable(["Class_name", "Samples_number", "Label"])
    table.align["Class_name"] = "l"
    table.padding_width = 1
    for i in range(nb_classes):
        table.add_row([class_name[i], class_list[i], i])
    print(table)
    print("Generated data_size is", data.shape)
    print("Generated labels_size is", labels.shape)
    return data, labels


Using TensorFlow backend.


In [2]:
from keras.models import Sequential 
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras.preprocessing.image import load_img , img_to_array
from keras.callbacks import CSVLogger
from keras.utils import np_utils
from keras.utils.visualize_util import plot
from keras.applications.vgg16 import VGG16, preprocess_input
from sklearn.cross_validation import train_test_split, cross_val_score, KFold
from prettytable import PrettyTable
import numpy as np
import os

In [3]:
data, labels = generate_data("H:\surfzjy\workspace\keras_study\practise\swimcat_data")

784 samples in 5 categories
The shape of each sample is (125, 125, 3)
+---------------+----------------+-------+
| Class_name    | Samples_number | Label |
+---------------+----------------+-------+
| A-sky         |      224       |   0   |
| B-pattern     |       89       |   1   |
| C-thick-dark  |      251       |   2   |
| D-thick-white |      135       |   3   |
| E-veil        |       85       |   4   |
+---------------+----------------+-------+
Generated data_size is (784, 125, 125, 3)
Generated labels_size is (784,)


In [4]:
data /= 255
test_size_ratio=0.20
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=test_size_ratio)

In [5]:
model = VGG16(weights='imagenet', include_top=False)

In [6]:
train_bnfeature = model.predict(train_data)
test_bnfeature = model.predict(test_data)

In [8]:
np.save(open('train_bnfeature.npy', 'wb'), train_bnfeature)
np.save(open('test_bnfeature.npy', 'wb'), test_bnfeature)

In [11]:
train_labels = np_utils.to_categorical(train_labels, nb_classes=5)
test_labels = np_utils.to_categorical(test_labels, nb_classes=5)

In [14]:
train_data = np.load(open('train_bnfeature.npy','rb'))
test_data = np.load(open('test_bnfeature.npy','rb'))

In [15]:
model = Sequential()
model.add(Flatten(input_shape=train_bnfeature.shape[1:]))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
model.fit(train_data, train_labels, 
          nb_epoch=50, batch_size=16)

Epoch 1/50
627/627 [==============================] - 1s - loss: 1.3701 - acc: 0.4721     
Epoch 2/50
627/627 [==============================] - 0s - loss: 0.8279 - acc: 0.6826     
Epoch 3/50
627/627 [==============================] - 0s - loss: 0.5560 - acc: 0.8293     
Epoch 4/50
627/627 [==============================] - 0s - loss: 0.4618 - acc: 0.8453     
Epoch 5/50
627/627 [==============================] - 0s - loss: 0.3616 - acc: 0.8740     
Epoch 6/50
627/627 [==============================] - 0s - loss: 0.2892 - acc: 0.8947     
Epoch 7/50
627/627 [==============================] - 0s - loss: 0.2602 - acc: 0.9203     
Epoch 8/50
627/627 [==============================] - 0s - loss: 0.2472 - acc: 0.9011     
Epoch 9/50
627/627 [==============================] - 0s - loss: 0.2107 - acc: 0.9362     
Epoch 10/50
627/627 [==============================] - 0s - loss: 0.1759 - acc: 0.9346     
Epoch 11/50
627/627 [==============================] - 0s - loss: 0.1758 - acc: 0.9330   

In [17]:
score = model.evaluate(test_data, test_labels, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.955414012739
